## Assignment 3 Question 2

Is it possible to predict ELO of a player based on context of a potential en passant move?

Only investigating the player who is in the position of making the en passant move.

Lila See FDS PCA solutions for 3 good points for what PCA is good for - include in report?

#### Import Libraries

In [1]:
import pandas as pd
import chess
import chess.pgn
import io
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import time
import stockfish

# PCA
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

#### Import CSV file

66879 entries in dataframe

In [2]:
# Load CSV into a big dataframe
CHESS_DATA_LOCATION = "data/club_games_data.csv"
chess_data = pd.read_csv(CHESS_DATA_LOCATION)

#### Dataframe info

Using:
print(chess_data.dtypes)
print(chess["rules"].unique())


Chess rules:
['chess' 'chess960' 'threecheck' 'crazyhouse' 'kingofthehill']

Time control:
['1/259200' '1/172800' '1800' '1/86400' '1/432000' '1/604800' '600'
 '120+1' '900+10' '300' '180+2' '3600+5' '2700+45' '3600' '1/1209600'
 '180' '600+10' '60' '480+3' '300+5' '420+3' '600+5' '600+2' '1200' '30'
 '60+1' '120' '1500+3' '900+2' '1500+5' '1500+10' '1/864000' '900' '300+2'
 '1500' '7200' '300+1' '5400' '3600+60' '2700+30' '3480+45' '10' '2700+10'
 '15' '2700' '3600+20' '4500' '4200' '900+5' '1800+10' '2700+5' '480+5'
 '1800+30' '300+3' '600+1' '1800+5' '420+5' '5400+30' '240+10' '420' '303'
 '60+10']

 Time class:
['daily' 'rapid' 'bullet' 'blitz']

#### Clean data

Undeveloped board shouldn't matter if we're filtering games for potential ep

Same for draws

Can filter our time class if are looking at time controls

After making a new move_list column, should we drop the pgn column?


Variables we are considering when predicting ELO (for a player who could potentially make en passant move) are: (Y = DONE, N = NOT DONE)
- Y: Colour who had ep opportunity (boolean)
- Y: Did they take the en passant? (boolean)
- N: Does their choice on taking/not taking support them if gaining an advantage? (numerical value for how much of an advantage it gives)
- N: Time taken to decide to capture/not capture en passant (... whatever can be time, a number in seconds ig)
- Y: Is the game rated? (boolean)
- Y: Game time class


To do:
- Work out which colour is making the potential en passant move, add a column to dataframe detailing this
- Make dataframe columns for other variables
- Apply PCA reduction

In [3]:
# Drops rows if any value is a NaN (data is clean so it doesn't do anything)
chess_data.dropna(axis=0, how='any')

# Filter out alternative rules like chess960 etc
# Important this comes before the drop irrelevant columns line
chess_data = chess_data[chess_data['rules'] == "chess"]

# Save PGN column from dataframe
full_pgn = chess_data['pgn']

def get_moves(entry):
    '''
    Retrive series of moves in a game when given the whole full_pgn entry
    '''
    pgn = entry.splitlines()[-1]
    return pgn

# Add list of moves (string) as a new column to dataframe
chess_data['move_list'] = full_pgn.apply(get_moves)

# Drop irrelevant columns
chess_data = chess_data.drop(['time_control', 'white_username', 'black_username','white_id', 'black_id', 'white_result', 'black_result', 'rules'], axis=1)




#### Import chess info
https://python-chess.readthedocs.io/en/latest/core.html#chess.Board.san


#### En Passant functions
- has_legal_en_passant() tests if en passant capturing would actually be possible on the next move.
- has_pseudo_legal_en_passant()
- has_legal_en_passant()
- is_en_passant(move: Move) Checks if the given pseudo-legal move is an en passant capture.




Use StringIO to parse games from a string.

```python
import io
pgn = io.StringIO("1. e4 e5 2. Nf3 *")
game = chess.pgn.read_game(pgn)
```

#### Clean data specifically for ep

Filter out columns that don't have potential ep
Add columns for: whether ep happened, which colour had potential to take ep

In [4]:
def check_pgn_opportunity(pgn_in):
    '''
    Checks PGN for whether opportunity for EP happened in the game.
    '''
    # print(pgn_in.splitlines()[-1])
    pgn = io.StringIO(pgn_in)       # PGN as a file
    game = chess.pgn.read_game(pgn) # Read PGN and put into game
    board = game.board()            # "board" of a game
    
    precheck = False                # Is en passant possible

    # Find only pawn moves in game
    for move in game.mainline_moves():
        # B bishop Q queen K king N knight R rook O castle ELSE pawn
        san = board.san(move)
        move_piece = san[0]
        
        # lestring = "abcdefg"
        # if [e in lestring for e in [move_piece] if e in lestring]:
        # if move_piece.startswith(("a", "b", "c", "d", "e", "f", "g")):
        # if (re.search("^[abcdefg]", move_piece) != None):
        if move_piece in "abcdefg":
            # Push the move before checking the board
            board.push(move)
            precheck = board.has_legal_en_passant()

            # Return True the moment a potential ep move has been found
            if precheck:
                return True
        else:
            board.push(move)
            continue
                
    # If ran through every move without ep opportunity then return False
    return False

def check_pgn_happened(pgn_in):
    '''
    Checks PGN for whether EP actually happened in the game.

    Assumes game passed in already has the opportunity for an EP move.
    Returns True if EP was actually the next move.
    '''
    pgn = io.StringIO(pgn_in)       # PGN as a file
    game = chess.pgn.read_game(pgn) # Read PGN and put into game
    board = game.board()            # "board" of a game
    
    moved = False                   # Was en passant moved?

    # Find only pawn moves in game
    for move in game.mainline_moves():
        # B bishop Q queen K king N knight R rook O castle ELSE pawn
        san = board.san(move)
        move_piece = san[0:1]

        if (re.search("[abcdefg]x", move_piece) != None):
            moved = board.is_en_passant(move)
            # If ep actually happened, immediately return True
            if moved:
                return True
            board.push(move)
        else:
            board.push(move)
            continue
                
    # If ran through every move without ep then return False
    return False

def check_pgn_turn(pgn_in):
    '''
    Checks which player has the opportunity for EP.

    Assumes check_pgn_opportunity is true for this pgn - otherwise None is returned.
    Returns 'White' or 'Black' depending on which player has opportunity for ep
    '''
    pgn = io.StringIO(pgn_in)       # PGN as a file
    game = chess.pgn.read_game(pgn) # Read PGN and put into game
    board = game.board()            # "board" of a game
    
    precheck = False                # Is en passant possible

    # Find only pawn moves in game
    for move in game.mainline_moves():
        # B bishop Q queen K king N knight R rook O castle ELSE pawn
        turn = board.turn
        san = board.san(move)
        move_piece = san[0]

        if move_piece in "abcdefg":
            # Push the move before checking the board
            board.push(move)
            precheck = board.has_legal_en_passant()

            # Return True the moment a potential ep move has been found
            if precheck:
                if turn:
                    return 'Black'
                else:
                    return 'White'
        else:
            board.push(move)
            continue
    # If pgn without ep opportunity was passed in, return None
    return None


def check_pgn(df_row):
    '''
    Checks PGN for whether opportunity for EP happened in the game.
    '''
    
    pgn_file = io.StringIO(df_row["pgn"])   # PGN as a file
    game = chess.pgn.read_game(pgn_file)    # Read PGN and put into game
    board = game.board()                    # "board" of a game
    # pre-initialise df_row values
    precheck = False
    moved = False
    move_colour = ""
    
    # comparison sets when checking move piece
    ep_set = set(["ax","bx","cx","dx","ex","fx","gx", "hx"])
    check_set = set("abcdefgh")
    
    # Find only pawn moves in game
    for move in game.mainline_moves():
        san = board.san(move)
        turn = board.turn
        move_piece = san[0]
        
        # check if precheck was flagged in previous move
        if precheck:
            # get first 2 letters, and compare to a set
            move_ep_piece = san[:2]
            if move_ep_piece in ep_set:
                # check if e.p. actually happened
                moved = board.is_en_passant(move)
            # break out for loop, no need to check further
            break
        
        # if the move was a pawn (lower case)
        if move_piece in check_set:
            # Push the move before checking the board
            board.push(move)
            # check if the next move can be en passant
            precheck = board.has_legal_en_passant()

            # if precheck is true then set the turn
            if precheck:
                if turn:
                    move_colour = 'Black'
                else:
                    move_colour = 'White'
        # if the move wasn't a pawn then just continue as normal
        else:
            board.push(move)
    
    # set row values
    df_row["ep_opportunity"] = precheck
    df_row["ep_happened"] = moved
    df_row["ep_colour"] = move_colour
    return df_row

# copy (so doesn't modify original dataframe)
chess_filter = chess_data.copy(deep=True)

start = time.time()
# apply e.p. finding function
chess_filter = chess_filter.apply(check_pgn, axis=1)
end = time.time()
print(end - start)

chess_filter = chess_filter[chess_filter['ep_opportunity'] == True]
chess_filter = chess_filter.drop(['ep_opportunity'], axis=1)
chess_filter = chess_filter[chess_filter['time_class'] != "daily"]

457.08684515953064


In [5]:
chess_filter3 = chess_filter[chess_filter['ep_happened'] == True]
chess_filter3 = chess_filter3.drop(['ep_happened'], axis=1)
print(len(chess_filter))
print(len(chess_filter3))
# print(chess_filter.head(50))

5366
1673


In [6]:
print(chess_filter.values)

[[1569 1546 'daily' ...
  '1. e4 c5 2. Bc4 e6 3. e5 Nc6 4. Nf3 d5 5. exd6 Bxd6 6. Bb5 Ne7 7. d4 a6 8. Ba4 Qa5+ 9. Nc3 b5 10. dxc5 Bxc5 11. Bb3 O-O 12. Bd2 Qc7 13. Ne4 Bb6 14. O-O Rd8 15. Nfg5 Nd5 16. Qh5 h6 17. Nh3 Qe5 18. Qg4 Nf6 19. Nxf6+ Qxf6 20. Rad1 e5 21. Qg3 Nd4 22. Rde1 Nxb3 23. axb3 Rxd2 24. Rxe5 Bxh3 25. gxh3 Bc7 26. Qe3 Qxe5 27. Qxe5 1-0'
  True 'White']
 [1505 1635 'daily' ...
  '1. d4 Nf6 2. c4 g6 3. Nc3 Bg7 4. e4 d6 5. Nf3 O-O 6. Bg5 Nbd7 7. Bd3 c5 8. d5 e5 9. dxe6 fxe6 10. Qd2 Qe7 11. O-O a6 12. a4 Qf7 13. Rfe1 Nb6 14. a5 Nbd7 15. Bc2 Nh5 0-1'
  True 'White']
 [1468 1870 'daily' ...
  '1. d4 c5 2. d5 e5 3. e4 d6 4. Nc3 Nf6 5. Nf3 Bg4 6. h3 Bh5 7. Bb5+ Nbd7 8. Bg5 Be7 9. Bxf6 Bxf6 10. a4 a6 11. Be2 O-O 12. Nd2 Bxe2 13. Qxe2 Bg5 14. Nc4 f5 15. Nxd6 fxe4 16. Ndxe4 Bh6 0-1'
  False 'White']
 ...
 [851 862 'blitz' ...
  '1. e4 {[%clk 0:05:00]} 1... e5 {[%clk 0:04:58.3]} 2. Nf3 {[%clk 0:04:59.3]} 2... d6 {[%clk 0:04:55.9]} 3. d4 {[%clk 0:04:57]} 3... c5 {[%clk 0:04:53.5]} 4. d

#### Cleaning data for EP - Advantage for ep

Analysing game - finding if ep move would have given an edvantage


Split into cases: ep happened and ep didn't happen


NOT DONE YET

In [7]:
def get_advantage(row):
    # make board
    # find move with e.p.
    # stockfish eval at that point
    # get stockfish eval if best move
    # get stockfish eval if e.p.
    # find difference between two
    # return

    ep_set = set(["ax","bx","cx","dx","ex","fx","gx", "hx"])
    check_set = set("abcdefgh")

    pgn_file = io.StringIO(row["pgn"])   # PGN as a file
    game = chess.pgn.read_game(pgn_file)    # Read PGN and put into game
    board = game.board()                    # "board" of a game
    
    for move in game.mainline_moves():
        san = board.san(move)
        turn = board.turn
        move_piece = san[0]
        
        # check if precheck was flagged in previous move
        if precheck:
            # get first 2 letters, and compare to a set
            move_ep_piece = san[:2]
        
        # if the move was a pawn (lower case)
        if move_piece in check_set:
            # Push the move before checking the board
            board.push(move)
            # check if the next move can be en passant
            precheck = board.has_legal_en_passant()
        # if the move wasn't a pawn then just continue as normal
        else:
            board.push(move)


    row["state_ep"] # stockfish eval if e.p. is taken
    row["state_no_ep"] # stockfish eval if e.p. isn't taken
    return row

chess_filter = chess_filter.apply(get_advantage)


KeyError: 'pgn'

### Number of games with at least 1 possible en-passant move - ???:
## 4750
(maybe 4913 or even 4945)

### Number of games with an actual en-passant move - ???:
## 1566

#### Applying PCA


In [ ]:
# Replace Boolean and string variables with numbers
chess_filter['ep_happened'] = chess_filter['ep_happened'].replace({True:1, False:0})
chess_filter['ep_colour'] = chess_filter['ep_colour'].replace({'White':1, 'Black':0})
chess_filter['rated'] = chess_filter['rated'].replace({True:1, False:0})
chess_filter['time_class'] = chess_filter['time_class'].replace({'daily':3, 'rapid':2, 'blitz':1, 'bullet':0})

# Drop irrelevant columns, and save differently - as a dataframe including the rating and one without
chess_data_with_elo = chess_filter.drop(['fen', 'pgn', 'move_list'], axis=1)
chess_data_without_elo = chess_data_with_elo.drop(['white_rating', 'black_rating'], axis=1)

print(chess_data_with_elo.head())
print('\n')
print(chess_data_without_elo.head())

    white_rating  black_rating  time_class  rated  ep_happened  ep_colour
40          1569          1546           3      1            0          1
48          1505          1635           3      1            0          1
55          1468          1870           3      1            0          1
56          1530          1459           3      1            0          0
81          1498          1540           2      1            0          1


    time_class  rated  ep_happened  ep_colour
40           3      1            0          1
48           3      1            0          1
55           3      1            0          1
56           3      1            0          0
81           2      1            0          1


##### Data Standardisation (For PCA)

Cite week 8 lab sheet in report? Heavily used their code to help

In [ ]:
# THIS PART IS COPIED AND PASTED FROM WEEK 8 PCA SOLUTIONS:
def sort_eigenvalues(eigenvalues, eigenvectors):
    idx = eigenvalues.argsort()[::-1]   
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:,idx]
    return eigenvalues, eigenvectors

# THE REST IS HEAVILY INFLUENCED BY WEEK 8 PCA SOLUTIONS

# Standardise chess_data_without_elo so that it follows the Standard normal distribution
for col in chess_data_without_elo.columns:
        chess_data_without_elo[col] = (chess_data_without_elo[col] - chess_data_without_elo[col].mean()) / chess_data_without_elo[col].std()

# # Create matrix storing covariances of the chess dataframe features
# covariances = np.cov(chess_data_without_elo.values.T)

# # Record eigenvalues and eigenvectors
# eigenvalues, eigenvectors = np.linalg.eig(covariances)
# eigenvalues, eigenvalues = sort_eigenvalues(eigenvalues, eigenvectors)

# result = chess_data_without_elo.dot(eigenvectors[:,:2])

# sns.scatterplot(x= result[0], y = result[1], hue=diagnosis)
# plt.xlabel('PC1')
# plt.ylabel('PC2')




# Lila Can we instead use this? :
pca = PCA(n_components=3).fit(chess_data_without_elo.values)
pca_result = pca.transform(chess_data_without_elo.values)
sns.scatterplot(x=pca_result[:, 0], y=pca_result[:, 1])
plt.xlabel('PC1')
plt.ylabel('PC2')

ValueError: Input X contains NaN.
PCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values